<a href="https://colab.research.google.com/github/aknip/llm-benchmark/blob/main/llm_simple_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install pandas openpyxl

# 1. Create Excel file

In [1]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# see https://openpyxl.readthedocs.io/en/2.4/pandas.html

# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Name': ['Miller', 'Adams', 'Smith'],
                   'Prompt': ['Write something', 'Do different things', 'Summarize it'], })

# Create a Pandas Excel writer using openpyxl as the engine.
writer = pd.ExcelWriter('llm_benchmark.xlsx', engine='openpyxl')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Benchmark', index=True)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Benchmark']

# Save the Excel file.
workbook.save('llm_benchmark.xlsx')

# Alternate approach without ExcelWriter:
# workbook = Workbook()
# worksheet = workbook.active
# for row in dataframe_to_rows(df, index=True, header=True):
#     if row != [None]:
#       worksheet.append(row)

# 2. Read Excel file as dataframe

In [17]:
import pandas as pd
#import xlrd

df = pd.read_excel('llm_benchmark.xlsx', sheet_name='Benchmark') # parse_dates=['date'] # dtype={'column_name': float}
#print(df.head())

print("Given Dataframe :\n", df)

print("\nIterating 1:")
for i in df.index:
    print(i, df['Name'][i],df['Prompt'][i])

print("\nIterating 2:")
for index, row in df.iterrows():
    name = row['Name']
    prompt = row['Prompt']
    print(f"{index}: {name}, {prompt}")

Given Dataframe :
    Unnamed: 0    Name               Prompt
0           0  Miller      Write something
1           1   Adams  Do different things
2           2   Smith         Summarize it

Iterating 1:
0 Miller Write something
1 Adams Do different things
2 Smith Summarize it

Iterating 2:
0: Miller, Write something
1: Adams, Do different things
2: Smith, Summarize it


# 3. Updata data and Excel file

In [18]:
df.at[1, 'Name'] = 'Name-Updated'
df.at[2, 'Prompt'] = 'Prompt-Updated'
print(df.head())

   Unnamed: 0          Name               Prompt
0           0        Miller      Write something
1           1  Name-Updated  Do different things
2           2         Smith       Prompt-Updated


In [19]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
workbook = load_workbook(filename = 'llm_benchmark.xlsx')
worksheet = workbook.active

print("Updating Excel file:")
for index, row in df.iterrows():
    name = row['Name']
    prompt = row['Prompt']
    print(f"{index}: {name}, {prompt}")
    d = worksheet.cell(row=index+2, column=2, value=name)
    d = worksheet.cell(row=index+2, column=3, value=prompt)

# Alternative: delete data rows and append dataframe to worksheet
# continuously delete row 2 until there is only first row (header row) is left over
# while(worksheet.max_row > 1):
#     worksheet.delete_rows(2) # removes the row 2
# append dataframe to worksheet
# for row in dataframe_to_rows(df, index=False, header=False):
#     if row != [None]:
#        worksheet.append(row)

# Save the Excel file.
workbook.save('llm_benchmark.xlsx')

Updating Excel file:
0: Miller, Write something
1: Name-Updated, Do different things
2: Smith, Prompt-Updated
